## Simple Mission in Guided-Mode

This notebook serves to get familiar with the simulator

In [1]:
import os
import time

from config import (
    ARDU_LOGS_PATH,
    ARDUPILOT_VEHICLE_PATH,
    ENV_CMD_PYT,
    VEH_PARAMS_PATH,
    BasePort,
)
from helpers import clean, create_process, setup_logging
from plan import State
from plan.planner import Plan
from proxy import create_tcp_conn

clean()

## Launch Copter (ardupilot)

In [2]:
#This must agree with first waypoint in mission.waypoint
spawn_str= '-35.3633245,149.1652241,0.0,0' 

sim_vehicle_path = os.path.expanduser(ARDUPILOT_VEHICLE_PATH)
sysid = 1
vehicle_cmd = (
    f"python3 {sim_vehicle_path} "
    f"-v ArduCopter "
    f"-I{sysid-1} --sysid {sysid} "
    f"--no-rebuild "
    f"--use-dir={ARDU_LOGS_PATH} "
    f"--add-param-file {VEH_PARAMS_PATH} "
    f"--no-mavproxy "
    f"--port-offset=0 "  # ArduPilot automatically binds TCP port 5760 (BasePort.ARP)
    f'--custom-location={spawn_str}'
)

create_process(
                vehicle_cmd,
                after="exec bash",
                visible= True,
                suppress_output=False,
                title="ardu_vehicle",
                env_cmd=ENV_CMD_PYT,
            )

<Popen: returncode: None args: ['xterm', '-T', 'ardu_vehicle', '-geometry', ...>

## Connect to the vehicle

In [3]:
while True:
    try:
        conn = create_tcp_conn(
            base_port=BasePort.ARP, 
            offset=0,
            role="client", 
            sysid=sysid
        )
        break 
    except Exception:
        print("⏳ Waiting for ArduPilot TCP connection...")
        time.sleep(0.5)

print("✅ TCP connection established!")

[Errno 111] Connection refused sleeping
[Errno 111] Connection refused sleeping


ERROR:root:Failed to create TCP connection on port 5760: [Errno 111] Connection refused


⏳ Waiting for ArduPilot TCP connection...
[Errno 111] Connection refused sleeping
[Errno 111] Connection refused sleeping


ERROR:root:Failed to create TCP connection on port 5760: [Errno 111] Connection refused


⏳ Waiting for ArduPilot TCP connection...
[Errno 111] Connection refused sleeping
[Errno 111] Connection refused sleeping


...

⏳ Waiting for ArduPilot TCP connection...
[Errno 111] Connection refused sleeping
[Errno 111] Connection refused sleeping


ERROR:root:Failed to create TCP connection on port 5760: [Errno 111] Connection refused


⏳ Waiting for ArduPilot TCP connection...
[Errno 111] Connection refused sleeping
[Errno 111] Connection refused sleeping


ERROR:root:Failed to create TCP connection on port 5760: [Errno 111] Connection refused


⏳ Waiting for ArduPilot TCP connection...
✅ TCP connection established!


## Create Plan in guided mode

Action commands are sent online, in contrast to auto mode where the full plan is loaded into the UAV before starting the mission.

In [4]:
plan = Plan.square(side_len=5, alt=5, wp_margin=0.5, navegation_speed=5)

In [5]:
plan

🕓 <Plan '📋 Square Trajectory'>
  🕓 <Action '🔧 PREARM'>
    🕓 <Step '🔹 Check disarmed'>
    🕓 <Step '🔹 Check EKF status'>
    🕓 <Step '🔹 Check GPS'>
    🕓 <Step '🔹 Check system'>
  🕓 <Action '⚙️  MODE: GUIDED'>
    🕓 <Step '🔹 Switch to GUIDED'>
  🕓 <Action '🔐 ARM'>
    🕓 <Step '🔹 arm'>
  🕓 <Action '🛫 TAKEOFF'>
    🕓 <Step '🔹 takeoff'>
  🕓 <Action '🛩️ FLY'>
    🕓 <Step '🔹 go to  -> (0, 0, 5)'>
    🕓 <Step '🔹 go to  -> (0, 5, 5)'>
    🕓 <Step '🔹 go to  -> (5, 5, 5)'>
    🕓 <Step '🔹 go to  -> (5, 0, 5)'>
    🕓 <Step '🔹 go to  -> (0, 0, 5)'>
  🕓 <Action '🛬 LAND'>
    🕓 <Step '🔹 land'>

# Execute Plan in guided mode

In [6]:
plan.bind(conn)
setup_logging('plan',verbose=2)
while plan.state != State.DONE:
    plan.act()

11:34:01 - plan - DEBUG - ▶️ Vehicle 1: Plan Started: 📋 Square Trajectory
11:34:01 - plan - DEBUG - ▶️ Vehicle 1: Action Started: 🔧 PREARM
11:34:01 - plan - DEBUG - ▶️ Vehicle 1: Step Started: Check disarmed
11:34:01 - plan - DEBUG - ✅ Vehicle 1: Step Done: Check disarmed
11:34:01 - plan - DEBUG - Vehicle 1: 📡 Requested message EKF_STATUS_REPORT at 1.00 Hz
11:34:01 - plan - DEBUG - ▶️ Vehicle 1: Step Started: Check EKF status
11:34:04 - plan - DEBUG - 🛰️ Vehicle 1: Waiting for EKF to be ready... Pending: ATTITUDE, VELOCITY_HORIZ, POS_VERT_ABS, POS_HORIZ_ABS
11:34:05 - plan - DEBUG - 🛰️ Vehicle 1: Waiting for EKF to be ready... Pending: ATTITUDE, VELOCITY_HORIZ, POS_VERT_ABS, POS_HORIZ_ABS
11:34:06 - plan - DEBUG - 🛰️ Vehicle 1: Waiting for EKF to be ready... Pending: POS_HORIZ_ABS
11:34:07 - plan - DEBUG - 🛰️ Vehicle 1: Waiting for EKF to be ready... Pending: POS_HORIZ_ABS
11:34:08 - plan - DEBUG - 🛰️ Vehicle 1: Waiting for EKF to be ready... Pending: POS_HORIZ_ABS
11:34:09 - plan - DE

In [7]:
print(plan)

✅ <Plan '📋 Square Trajectory'>
  ✅ <Action '🔧 PREARM'>
    ✅ <Step '🔹 Check disarmed'>
    ✅ <Step '🔹 Check EKF status'>
    ✅ <Step '🔹 Check GPS'>
    ✅ <Step '🔹 Check system'>
  ✅ <Action '⚙️  MODE: GUIDED'>
    ✅ <Step '🔹 Switch to GUIDED'>
  ✅ <Action '🔐 ARM'>
    ✅ <Step '🔹 arm'>
  ✅ <Action '🛫 TAKEOFF'>
    ✅ <Step '🔹 takeoff'>
  ✅ <Action '🛩️ FLY'>
    ✅ <Step '🔹 go to  -> (0, 0, 5)'>
    ✅ <Step '🔹 go to  -> (0, 5, 5)'>
    ✅ <Step '🔹 go to  -> (5, 5, 5)'>
    ✅ <Step '🔹 go to  -> (5, 0, 5)'>
    ✅ <Step '🔹 go to  -> (0, 0, 5)'>
  ✅ <Action '🛬 LAND'>
    ✅ <Step '🔹 land'>


In [8]:
plan.reset()

In [9]:
print(plan)

🕓 <Plan '📋 Square Trajectory'>
  🕓 <Action '🔧 PREARM'>
    🕓 <Step '🔹 Check disarmed'>
    🕓 <Step '🔹 Check EKF status'>
    🕓 <Step '🔹 Check GPS'>
    🕓 <Step '🔹 Check system'>
  🕓 <Action '⚙️  MODE: GUIDED'>
    🕓 <Step '🔹 Switch to GUIDED'>
  🕓 <Action '🔐 ARM'>
    🕓 <Step '🔹 arm'>
  🕓 <Action '🛫 TAKEOFF'>
    🕓 <Step '🔹 takeoff'>
  🕓 <Action '🛩️ FLY'>
    🕓 <Step '🔹 go to  -> (0, 0, 5)'>
    🕓 <Step '🔹 go to  -> (0, 5, 5)'>
    🕓 <Step '🔹 go to  -> (5, 5, 5)'>
    🕓 <Step '🔹 go to  -> (5, 0, 5)'>
    🕓 <Step '🔹 go to  -> (0, 0, 5)'>
  🕓 <Action '🛬 LAND'>
    🕓 <Step '🔹 land'>
